In [1]:
import sys
import os
sys.path.insert(0, "../../python")

import numpy as np
from beam.beams import electronbeam
from beam.elements import plasma
import matplotlib.pyplot as plt
from beam import interactions
from ionization import ionization
from lens import profile

path = '/mnt/md0/Data/Tests/Beam_Scaling/'

In [2]:
# Now we can create a very simple plasma and propagate the beam through it
Nx = 50
Ny = 50
Nz = 256
def dgammadz(ne):
    return 0.0
plasmaParams ={
    'name' : 'TestPlasma',
    'path' : path,
    'load' : False,
    'Nx' : Nx,
    'Ny' : Ny,
    'Nz' : Nz,
    'X' : 100,
    'Y' : 100,
    'Z' : 2e6,
    'n0' : 0.1,
    'atom' : ionization.Ar,
    'cyl' : False,
    'dgammadz' : dgammadz
}

argon = plasma.Plasma(plasmaParams)
n = plasmaParams['n0']*np.ones((Nx, Ny, Nz), dtype='double')
ne = np.zeros((Nx, Ny, Nz), dtype='double')
ne[int(Nx/2), int(Ny/2), :] = plasmaParams['n0']*profile.plasma_gaussian_ramps(0.75e6, 0.5e6, 12e4, 12e4, Nz,
                                                                               plasmaParams['Z'])[1]
argon.initialize_plasma(n, ne)

In [3]:
electronParams = {
    'name' : 'TestBeam',
    'path' : path,
    'load' : False,
    'N' : 100000,
    'gamma' : 20000,
    'emittance' : 5e-6,
    'betax' : 0.10,
    'betay' : 0.10,
    'alphax' : 0.0,
    'alphay' : 0.0,
    'sigmaz' : 5e-6,
    'dE' : 0.01
}

In [4]:
m = 50
N = 50000
threads = np.linspace(0, 40, 11, dtype='int')
threads[0] = 1
print(threads)
dumpPeriod = N / m

[ 1  4  8 12 16 20 24 28 32 36 40]


In [5]:
for i in range(11):
    beam = electronbeam.GaussianElectronBeam(electronParams)
    interactions.electron_plasma(beam, argon, np.linspace(0, plasmaParams['Z'], N), dumpPeriod, threads[i])
    time = np.zeros(m, dtype='double')
    for j in range(0, m):
        time[j] = os.path.getmtime(path+'beams/beam_TestBeam/TestBeam_ptcls_'+str(j)+'.npy')
    print('Threads:', threads[i], 'Average:', np.average(np.diff(time)), 'Median:', np.median(np.diff(time)))

KeyboardInterrupt: 

In [ ]:
import cProfile
cProfile.run('interactions.electron_plasma(beam, argon, np.linspace(0, plasmaParams["Z"], N), dumpPeriod, 40)')